In [27]:
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects

import rpy2.robjects.numpy2ri
import pandas as pd

In [22]:
from DAX.HelpFunctions.get_dax_data import get_dax_data

df = get_dax_data()

In [23]:
y = df['ret1'].dropna().to_numpy()

# Setup Arma Garch 1,1

In [46]:

rpy2.robjects.numpy2ri.activate()


rugarch = rpackages.importr('rugarch')

bic_df = pd.DataFrame()

for p in range(1,6):
    for q in range(1,6):
        # GARCH(1,1)
        variance_model = robjects.ListVector({'model': "sGARCH",
                                              'garchOrder': robjects.IntVector([p, q])})
        # ARMA(1,1)
        mean_model = robjects.ListVector({'armaOrder': robjects.IntVector([0,0]),
                                          'include.mean': True})
        
        model = rugarch.ugarchspec(variance_model=variance_model,
                                       mean_model=mean_model,
                                       distribution_model="norm",  # "std" -> student t, "norm" -> normal
                                       )
        
        
        # In the following, I'm assuming that your timeseries is in a numpy array called "y"
        modelfit = rugarch.ugarchfit(spec=model,
                                     data=y,
                                     solver='hybrid',
                                     tol=1e-3)
        
        info_criteria = rugarch.infocriteria(modelfit)
        
        # Assuming the first two elements are AIC and BIC respectively
        aic = info_criteria[0]  # AIC
        bic = info_criteria[1][0]  # BIC
        
        # Print AIC and BIC
        print("AIC:", aic)
        print("BIC:", bic)
        print(f'{p} : {q} : {bic}')
        bic_df.loc[p,q] = bic
        
        # historic mu and sigma
        sigma_hist = np.asarray(rugarch.sigma(modelfit))
        mu_hist = np.asarray(rugarch.fitted(modelfit))
        
        # 1-day ahead forecast mu and sigma
        forecast = rugarch.ugarchforecast(modelfit)
        sigma = np.asarray(rugarch.sigma(forecast))
        mu = np.asarray(rugarch.fitted(forecast))

AIC: [3.18125534]
BIC: 3.1843685722711013
1 : 1 : 3.1843685722711013
AIC: [3.18197519]
BIC: 3.1858667288736666
1 : 2 : 3.1858667288736666
AIC: [3.18248713]
BIC: 3.187156969680764
1 : 3 : 3.187156969680764
AIC: [3.18265904]
BIC: 3.1881071940547643
1 : 4 : 3.1881071940547643
AIC: [3.18281423]
BIC: 3.1890406827796616
1 : 5 : 3.1890406827796616
AIC: [3.17852752]
BIC: 3.182419054886505
2 : 1 : 3.182419054886505
AIC: [3.1787461]
BIC: 3.183415941696546
2 : 2 : 3.183415941696546
AIC: [3.17907537]
BIC: 3.1845235161579786
2 : 3 : 3.1845235161579786
AIC: [3.17956741]
BIC: 3.1857938630780507
2 : 4 : 3.1857938630780507
AIC: [3.17961876]
BIC: 3.1866235216338867
2 : 5 : 3.1866235216338867
AIC: [3.17706937]
BIC: 3.181739214166251
3 : 1 : 3.181739214166251
AIC: [3.17665462]
BIC: 3.182102766655603
3 : 2 : 3.182102766655603
AIC: [3.17594466]
BIC: 3.1821711144363567
3 : 3 : 3.1821711144363567
AIC: [3.17595206]
BIC: 3.1829568258178274
3 : 4 : 3.1829568258178274
AIC: [3.17596616]
BIC: 3.183749231495095
3 : 

In [47]:
bic_df

,1,2,3,4,5
1,3.184369,3.185867,3.187157,3.188107,3.189041
2,3.182419,3.183416,3.184524,3.185794,3.186624
3,3.181739,3.182103,3.182171,3.182957,3.183749
4,3.182888,3.183270,3.183241,3.183926,3.184064
5,3.184121,3.184496,3.184509,3.185042,3.185061


In [48]:
sorted(np.array(bic_df).flatten())

[3.181739214166251,
 3.182102766655603,
 3.1821711144363567,
 3.182419054886505,
 3.1828880878983927,
 3.1829568258178274,
 3.183240609831,
 3.183269615333383,
 3.183415941696546,
 3.183749231495095,
 3.1839260016971953,
 3.1840636860879443,
 3.1841213228878607,
 3.1843685722711013,
 3.1844961719808507,
 3.184508844333391,
 3.1845235161579786,
 3.1850423497003724,
 3.1850606017272645,
 3.1857938630780507,
 3.1858667288736666,
 3.1866235216338867,
 3.187156969680764,
 3.1881071940547643,
 3.1890406827796616]

In [25]:
from DAX.Models import arma_garch_11 as ag11
import numpy as np
from DAX.HelpFunctions.get_dax_data import get_dax_data

import importlib
importlib.reload(ag11)

df = get_dax_data()
print(ag11.arma_garch_11_3df(df))
print(ag11.arma_garch_11_8df(df))

AttributeError: module 'DAX.Models.arma_garch_11' has no attribute 'arma_garch_11_3df'